# Tech Challenge - Fase 1: Predição de Ataque Cardíaco

## Análise de Dados e Machine Learning para Diagnóstico Médico

**Objetivo:** Criar um sistema de apoio ao diagnóstico médico que utilize técnicas de Machine Learning para estimar a probabilidade de ocorrência de um ataque cardíaco a partir de informações clínicas e demográficas de pacientes.

**Base de Dados:** Heart Attack Prediction - Indonesia

---

## 1. Introdução e Contexto do Problema

As doenças cardiovasculares continuam sendo uma das principais causas de morte em todo o mundo. Milhares de pessoas perdem a vida todos os anos em decorrência de eventos cardíacos que, em muitos casos, poderiam ser evitados com uma detecção precoce e um acompanhamento adequado.

Neste projeto, buscamos desenvolver um modelo preditivo capaz de identificar pacientes com maior risco de sofrer um ataque cardíaco. A proposta é aplicar técnicas de Machine Learning sobre dados clínicos estruturados, de forma a extrair padrões e indicadores relevantes que auxiliem no processo de triagem e tomada de decisão médica.

Ao automatizar parte dessa análise, espera-se reduzir o tempo necessário para a identificação de casos críticos e apoiar profissionais da saúde no planejamento de intervenções preventivas mais eficazes.

**Observação:** O sistema desenvolvido tem caráter complementar e serve apenas como ferramenta de apoio à decisão. O diagnóstico definitivo e as condutas clínicas devem ser sempre realizados por profissionais da área da saúde devidamente qualificados.


In [1]:
!pip3 install pandas numpy matplotlib seaborn scikit-learn shap

# Manipulação de dados
import pandas as pd
import numpy as np

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Exibição de dataframes
from IPython.display import display


ERROR: Could not find a version that satisfies the requirement warnings (from versions: none)
ERROR: No matching distribution found for warnings


## Exploração e análise de dados

#dados tecnicos, tamanho, colunas, quanto vai alocar de ram para eu trabalhar

In [ ]:
# Carregar o dataset
df = pd.read_csv('./dataset/heart_attack_prediction_indonesia.csv')

print("INFORMAÇÕES GERAIS DO DATASET")
print(f"Dimensões do dataset: {df.shape[0]} linhas x {df.shape[1]} colunas")
print(f"Memória utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print("
--- Informações do DataFrame ---")
df.info()

print("
--- Tipos de Dados ---")
print(df.dtypes)

print("
--- Primeiras 5 Linhas ---")
display(df.head())

print("
--- Últimas 5 Linhas ---")
display(df.tail())

print("
--- Contagem de Valores Nulos por Coluna ---")
print(df.isnull().sum())

print("
--- Porcentagem de Valores Nulos por Coluna ---")
print(df.isna().mean().sort_values(ascending=False))

print("
--- Estatísticas Descritivas (Todas as Colunas) ---")
display(df.describe(include='all'))

print("
--- Estatísticas Descritivas (Colunas Numéricas) ---")
display(df.describe())

print("
--- Estatísticas Descritivas (Colunas Categóricas) ---")
display(df.describe(include=['object', 'category']))
